In [ ]:
#Script to extract data from UFC website using web scraping.
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

def clean_record_text(record_text):
    # Clean up the record text if needed, for example, remove extra characters
    return record_text

def extract_numeric(text):
    # Extract numeric value from text
    match = re.search(r'[\d.]+', text)
    return float(match.group()) if match else 0

def get_fighter_details(fighter_url):
    """Scrape detailed information for a single fighter"""
    try:
        response = requests.get(f"https://www.ufc.com{fighter_url}")
        soup = BeautifulSoup(response.text, 'html.parser')

        # Basic info
        name = soup.select_one('h1.hero-profile__name')
        name = name.text.strip() if name else "Unknown"
        print(f"Name: {name}")

        weight_class = soup.select_one('.hero-profile__division-title')
        weight_class = weight_class.text.strip() if weight_class else "Unknown"
        print(f"Weight Class: {weight_class}")

        # Record
        record_text = soup.select_one('.hero-profile__division-body')

        if record_text:
            record_text = record_text.text.strip()
            record_text = clean_record_text(record_text)
        else:
            record_text = "0-0-0"
        print(f"Record Text: {record_text}")

        wins, losses, draws = 0, 0, 0
        record_parts = record_text.split('-')
        if len(record_parts) >= 3:
            try:
                wins = int(re.search(r'\d+', record_parts[0]).group())
            except (ValueError, AttributeError):
                wins = 0
            try:
                losses = int(re.search(r'\d+', record_parts[1]).group())
            except (ValueError, AttributeError):
                losses = 0
            try:
                draws = int(re.search(r'\d+', record_parts[2]).group())
            except (ValueError, AttributeError):
                draws = 0
        print(f"Wins: {wins}, Losses: {losses}, Draws: {draws}")

        # Physical stats
        height_elem = soup.find('div', text=re.compile(r'\bHeight\b', re.IGNORECASE))
        height = height_elem.find_next_sibling('div').text.strip() if height_elem else ""
        print(f"Height: {height}")

        weight_elem = soup.find('div', text=re.compile(r'\bWeight\b', re.IGNORECASE))
        weight = weight_elem.find_next_sibling('div').text.strip() if weight_elem else ""
        print(f"Weight: {weight}")

        reach_elem = soup.find('div', text=re.compile(r'\bReach\b', re.IGNORECASE))
        reach = reach_elem.find_next_sibling('div').text.strip() if reach_elem else ""
        print(f"Reach: {reach}")

        # Performance stats
        striking_accuracy_elem = soup.find('title', text=re.compile(r'\bStriking Accuracy\b', re.IGNORECASE))
        if striking_accuracy_elem:
            striking_accuracy_text = striking_accuracy_elem.text
            striking_accuracy = re.search(r'(\d+%)', striking_accuracy_text).group(1) if striking_accuracy_text else "0%"
        else:
            striking_accuracy = "0%"
        print(f"Striking Accuracy: {striking_accuracy}")

        strikes_landed = soup.find('div', text=re.compile(r'\bSig. Str. Landed\b', re.IGNORECASE))
        strikes_landed = extract_numeric(strikes_landed.find_next('div', class_='c-stat-compare__number').text) if strikes_landed else 0
        print(f"Strikes Landed: {strikes_landed}")

        strikes_absorbed = soup.find('div', text=re.compile(r'\bSig. Str. Absorbed\b', re.IGNORECASE))
        strikes_absorbed = extract_numeric(strikes_absorbed.find_next('div', class_='c-stat-compare__number').text) * 100 if strikes_absorbed else 0
        print(f"Strikes Absorbed: {strikes_absorbed}")

        strike_defense = soup.find('div', text=re.compile(r'\bSig. Str. Defense\b', re.IGNORECASE))
        strike_defense = extract_numeric(strike_defense.find_next('div', class_='c-stat-compare__number').text) if strike_defense else 0
        print(f"Strike Defense: {strike_defense}")

        takedown_avg = soup.find('div', text=re.compile(r'\bTakedown avg\b', re.IGNORECASE))
        takedown_avg = extract_numeric(takedown_avg.find_next('div', class_='c-stat-compare__number').text) if takedown_avg else 0
        print(f"Takedown Avg: {takedown_avg}")

        takedown_accuracy_elem = soup.find('title', text=re.compile(r'\bTakedown Accuracy\b', re.IGNORECASE))
        if takedown_accuracy_elem:
            takedown_accuracy_text = takedown_accuracy_elem.text
            takedown_accuracy = re.search(r'(\d+%)', takedown_accuracy_text).group(1) if takedown_accuracy_text else "0%"
        else:
            takedown_accuracy = "0%"
        print(f"Takedown Accuracy: {takedown_accuracy}")

        takedown_defense = soup.find('div', text=re.compile(r'\bTakedown Defense\b', re.IGNORECASE))
        takedown_defense = extract_numeric(takedown_defense.find_next('div', class_='c-stat-compare__number').text) if takedown_defense else 0
        print(f"Takedown Defense: {takedown_defense}")

        submission_avg = soup.find('div', text=re.compile(r'\bSubmission avg\b', re.IGNORECASE))
        submission_avg = extract_numeric(submission_avg.find_next('div', class_='c-stat-compare__number').text) if submission_avg else 0
        print(f"Submission Avg: {submission_avg}")

        avg_fight_time_elem = soup.find('div', text=re.compile(r'\bAverage fight time\b', re.IGNORECASE))
        avg_fight_time = avg_fight_time_elem.find_previous_sibling('div',
                                                                   class_='c-stat-compare__number').text.strip() if avg_fight_time_elem else "0"
        print(f"Average Fight Time: {avg_fight_time}")

        # KO/TKO percentage
        ko_tko_elem = soup.find('div', text=re.compile(r'\bKO/TKO\b', re.IGNORECASE))
        ko_tko_value = ko_tko_elem.find_next_sibling('div', class_='c-stat-3bar__value').text.strip() if ko_tko_elem else ""
        percentage_match = re.search(r'\((\d+%)\)', ko_tko_value)
        ko_tko_percentage = percentage_match.group(1) if percentage_match else "0%"
        print(f"KO/TKO Percentage: {ko_tko_percentage}")

        # Submissions percentage
        sub_elem = soup.find('div', text=re.compile(r'\bSUB\b', re.IGNORECASE))
        sub_value = sub_elem.find_next_sibling('div', class_='c-stat-3bar__value').text.strip() if sub_elem else ""
        percentage_sub = re.search(r'\((\d+%)\)', sub_value)
        sub_percentage = percentage_sub.group(1) if percentage_sub else "0%"
        print(f"Submission Percentage: {sub_percentage}")

        # Decision percentage
        dec_elem = soup.find('div', text=re.compile(r'\bDEC\b', re.IGNORECASE))
        dec_value = dec_elem.find_next_sibling('div', class_='c-stat-3bar__value').text.strip() if dec_elem else ""
        percentage_dec = re.search(r'\((\d+%)\)', dec_value)
        dec_percentage = percentage_dec.group(1) if percentage_dec else "0%"
        print(f"Decision Percentage: {dec_percentage}")

        # Significant strikes by standing
        sig_stand_elem = soup.find('div', text=re.compile(r'\bStanding\b', re.IGNORECASE))
        stand_value = sig_stand_elem.find_next_sibling('div', class_='c-stat-3bar__value').text.strip() if sig_stand_elem else ""
        percentage_stand = re.search(r'\((\d+%)\)', stand_value)
        stand_percentage = percentage_stand.group(1) if percentage_stand else "0%"
        print(f"Strikes By Standing: {stand_percentage}")

        # Significant strikes by clinch
        sig_clinch_elem = soup.find('div', class_='c-stat-3bar__label', text=re.compile(r'\bClinch\b', re.IGNORECASE))
        clinch_value = sig_clinch_elem.find_next_sibling('div',
                                                         class_='c-stat-3bar__value').text.strip() if sig_clinch_elem else ""
        percentage_clinch = re.search(r'\((\d+%)\)', clinch_value)
        clinch_percentage = percentage_clinch.group(1) if percentage_clinch else "0%"
        print(f"Strikes By Clinch: {clinch_percentage}")

        # Significant strikes by ground
        sig_ground_elem = soup.find('div', class_='c-stat-3bar__label', text=re.compile(r'\bGround\b', re.IGNORECASE))
        ground_value = sig_ground_elem.find_next_sibling('div',class_='c-stat-3bar__value').text.strip() if sig_ground_elem else ""
        percentage_ground = re.search(r'\((\d+%)\)', ground_value)
        ground_percentage = percentage_ground.group(1) if percentage_ground else "0%"
        print(f"Strikes By Ground: {ground_percentage}")

        # Significant strikes to head
        head_percent_elem = soup.find('text', {'id': 'e-stat-body_x5F__x5F_head_percent'})
        head_percentage = head_percent_elem.text.strip() if head_percent_elem else "0%"
        print(f"Strikes To Head: {head_percentage}")

        # Significant strikes to body
        body_percent_elem = soup.find('text', {'id': 'e-stat-body_x5F__x5F_body_percent'})
        body_percentage = body_percent_elem.text.strip() if body_percent_elem else "0%"
        print(f"Strikes To Body: {body_percentage}")

        # Significant strikes to legs
        leg_percent_elem = soup.find('text', {'id': 'e-stat-body_x5F__x5F_leg_percent'})
        leg_percentage = leg_percent_elem.text.strip() if leg_percent_elem else "0%"
        print(f"Strikes To Legs: {leg_percentage}")

        # Get fighter image
        image = soup.select_one('.hero-profile__image-wrap img')
        image_url = image['src'] if image and 'src' in image.attrs else ""
        print(f"Image URL: {image_url}")

        # Assemble fighter data
        fighter_data = {
            "id": fighter_url.split('/')[-1],
            "name": name,
            "weightClass": weight_class,
            "wins": wins,
            "losses": losses,
            "draws": draws,
            "height": height,
            "weight": weight,
            "reach": reach,
            "strikesLandedPerMin": strikes_landed,
            "strikingAccuracy": striking_accuracy,
            "strikesAbsorbedPerMin": strikes_absorbed,
            "strikeDefense": strike_defense,
            "takedownAvgPer15Min": takedown_avg,
            "takedownAccuracy": takedown_accuracy,
            "takedownDefense": takedown_defense,
            "submissionAvgPer15Min": submission_avg,
            "avgFightTime": avg_fight_time,
            "KoTkoPercentage": ko_tko_percentage,
            "SubmissionPercentage": sub_percentage,
            "DecisionPercentage": dec_percentage,
            "StrikesByStanding": stand_percentage,
            "StrikesByClinching": clinch_percentage,
            "StrikesByGround": ground_percentage,
            "StrikesToHead": head_percentage,
            "StrikesToBody": body_percentage,
            "StrikesToLegs": leg_percentage,
            "image": image_url
        }

        return fighter_data
        print('extracted data for',name,':',fighter_data)
    except Exception as e:
        print(f"Error scraping fighter {fighter_url}: {e}")
        return None

def scrape_fighters(target_count=500):
    """Scrape fighters from UFC website until target count is reached"""
    fighters = []
    page = 1

    while len(fighters) < target_count:
        try:
            url = f"https://www.ufc.com/athletes/all?page={page}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Correct selector for the fighter items
            fighter_items = soup.select('.c-listing-athlete-flipcard .c-listing-athlete-flipcard__inner .c-listing-athlete-flipcard__back .c-listing-athlete-flipcard__action')

            if not fighter_items:
                print(f"No fighter items found on page {page}")
                break

            for item in fighter_items:
                try:
                    # Correct selector for the profile link
                    link = item.select_one('a')
                    if link and 'href' in link.attrs:
                        fighter_url = link['href']

                        fighter_data = get_fighter_details(fighter_url)
                        if fighter_data:
                            fighters.append(fighter_data)
                            # Save progress after each fighter
                            with open('ufc_fighters.json', 'w') as f:
                                json.dump(fighters, f, indent=2)

                        # Be nice to the server
                        time.sleep(2)
                except Exception as e:
                    print(f"Error processing fighter item: {e}")
                    continue

            print(f"Completed page {page}")
            page += 1
            # Be nice to the server between pages
            time.sleep(5)

        except Exception as e:
            print(f"Error processing page {page}: {e}")
            break

    print(f"Scraped {len(fighters)} fighters successfully!")
    return fighters

if __name__ == "__main__":
    print("Starting UFC fighter scraper...")
    fighters = scrape_fighters(target_count=500)
    print(f"Completed scraping {len(fighters)} fighters.")

    # Convert the list of dictionaries to a DataFrame
    dataset = pd.DataFrame(fighters)
    print(dataset)

    # Save the DataFrame to a CSV file
    dataset.to_csv('fighters_data.csv', index=False)
    print("Data saved to fighters_data.csv")

Starting UFC fighter scraper...
Name: Shamil Abdurakhimov
Weight Class: Heavyweight Division
Record Text: 20-8-0 (W-L-D)
Wins: 20, Losses: 8, Draws: 0
Height: 75.00
Weight: 263.00
Reach: 76.00
Striking Accuracy: 44%
Strikes Landed: 3.02
Strikes Absorbed: 101.0
Strike Defense: 45.0
Takedown Avg: 0.14
Takedown Accuracy: 23%
Takedown Defense: 0.29
Submission Avg: 55.0
Average Fight Time: 09:27
KO/TKO Percentage: 45%
Submission Percentage: 20%
Decision Percentage: 35%
Strikes By Standing: 67%
Strikes By Clinch: 24%
Strikes By Ground: 9%
Strikes To Head: 59%
Strikes To Body: 19%
Strikes To Legs: 22%
Image URL: https://ufc.com/images/styles/athlete_bio_full_body/s3/2023-02/ABDURAKHIMOV_SHAMIL_L_02_07.png?itok=PHRwbOwY


<ipython-input-3-3416dff1d0c9>:62: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  height_elem = soup.find('div', text=re.compile(r'\bHeight\b', re.IGNORECASE))
<ipython-input-3-3416dff1d0c9>:66: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  weight_elem = soup.find('div', text=re.compile(r'\bWeight\b', re.IGNORECASE))
<ipython-input-3-3416dff1d0c9>:70: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  reach_elem = soup.find('div', text=re.compile(r'\bReach\b', re.IGNORECASE))
<ipython-input-3-3416dff1d0c9>:75: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  striking_accuracy_elem = soup.find('title', text=re.compile(r'\bStriking Accuracy\b', re.IGNORECASE))
<ipython-input-3-3416dff1d0c9>:83: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use '

Streaming output truncated to the last 5000 lines.
Decision Percentage: 29%
Strikes By Standing: 94%
Strikes By Clinch: 6%
Strikes By Ground: 0%
Strikes To Head: 59%
Strikes To Body: 31%
Strikes To Legs: 9%
Image URL: https://ufc.com/images/styles/athlete_bio_full_body/s3/2024-02/BRYZCEK_ROBERT_L_02-10.png?itok=A9mC-jwV
Name: Lukasz Brzeski
Weight Class: Heavyweight Division
Record Text: 9-6-1 (W-L-D)
Wins: 9, Losses: 6, Draws: 1
Height: 75.00
Weight: 234.00
Reach: 78.00
Striking Accuracy: 50%
Strikes Landed: 3.68
Strikes Absorbed: 43.0
Strike Defense: 37.0
Takedown Avg: 0.43
Takedown Accuracy: 33%
Takedown Defense: 0.0
Submission Avg: 55.0
Average Fight Time: 10:01
KO/TKO Percentage: 67%
Submission Percentage: 11%
Error scraping fighter /athlete/lukasz-brzeski: 'NoneType' object has no attribute 'text'
Name: Justin Buchholz
Weight Class: Lightweight Division
Record Text: 9-5-0 (W-L-D)
Wins: 9, Losses: 5, Draws: 0
Height: 72.00
Weight: 156.00
Reach: 
Striking Accuracy: 27%
Strikes Land

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import joblib
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets
import requests

# Load the CSV file into a DataFrame
dataset = pd.read_csv('fighters_data.csv')

# Handle missing values for descriptive columns
dataset['weightClass'] = dataset['weightClass'].fillna(dataset['weightClass'].mode().iloc[0])

# Convert percentage columns to numeric by removing '%' sign and dividing by 100
percentage_columns = ['strikingAccuracy', 'takedownAccuracy', 'KoTkoPercentage',
                      'SubmissionPercentage', 'DecisionPercentage', 'StrikesByStanding',
                      'StrikesByClinching', 'StrikesByGround', 'StrikesToHead', 'StrikesToBody',
                      'StrikesToLegs']

for col in percentage_columns:
    if col in dataset.columns:
        dataset[col] = dataset[col].str.replace('%', '').astype(float) / 100

# Convert avgFightTime from min:secs to seconds
def convert_time_to_seconds(time_str):
    try:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    except:
        return np.nan

if 'avgFightTime' in dataset.columns:
    dataset['avgFightTime'] = dataset['avgFightTime'].apply(convert_time_to_seconds)

# List of numeric columns to normalize
numeric_columns = [
    'height', 'weight', 'reach', 'strikesLandedPerMin', 'strikingAccuracy',
    'strikesAbsorbedPerMin', 'strikeDefense', 'takedownAvgPer15Min',
    'takedownAccuracy', 'takedownDefense', 'submissionAvgPer15Min',
    'avgFightTime', 'KoTkoPercentage', 'SubmissionPercentage',
    'DecisionPercentage', 'StrikesByStanding', 'StrikesByClinching',
    'StrikesByGround', 'StrikesToHead', 'StrikesToBody', 'StrikesToLegs',
    'wins', 'losses', 'draws'
]

# Convert other numeric columns to numeric, coercing errors to NaN
dataset[numeric_columns] = dataset[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Identify columns that are all NaN
all_nan_columns = dataset.columns[dataset.isna().all()].tolist()

# Drop columns that are all NaN
dataset = dataset.dropna(axis=1, how='all')

# Preserve original values before normalization
original_dataset = dataset.copy()

# Fill missing values in numeric columns with the mean
dataset[numeric_columns] = dataset[numeric_columns].fillna(dataset[numeric_columns].mean())

# Normalize the numeric columns using Min-Max scaling
scaler = MinMaxScaler()
dataset[numeric_columns] = scaler.fit_transform(dataset[numeric_columns])

# Select features and target variable for modeling
features = numeric_columns
# Assuming a placeholder target variable with int type for classification
dataset['target'] = (dataset['wins'] > dataset['losses']).astype(int)

# Split the data into features and target
X = dataset[features]
y = dataset['target']

# Initialize the Random Forest model
model = RandomForestClassifier(random_state=42)

# Train the model
model.fit(X, y)

# Save the trained model
joblib.dump(model, 'ufc_fight_predictor.pkl')

# Function to get fighter stats as per the dataset
def get_fighter_stats(fighter_name):
    fighter_name = fighter_name.strip().lower()
    fighter = original_dataset[original_dataset['name'].str.lower() == fighter_name]
    if not fighter.empty:
        return fighter.iloc[0], dataset[dataset['name'].str.lower() == fighter_name][features].values[0]
    return None, None

# Function to predict fight outcome
def predict_fight_outcome(fighter1_name, fighter2_name):
    fighter1_data, stats1 = get_fighter_stats(fighter1_name)
    fighter2_data, stats2 = get_fighter_stats(fighter2_name)
    if stats1 is not None and stats2 is not None:
        combined_stats = np.abs(stats1 - stats2)
        combined_stats_df = pd.DataFrame([combined_stats], columns=features)
        prediction = model.predict(combined_stats_df)[0]

        if prediction == 1:
            winner = fighter1_name
            win_method = determine_win_method(fighter1_data, fighter2_data)
        else:
            winner = fighter2_name
            win_method = determine_win_method(fighter2_data, fighter1_data)

        return winner, win_method
    else:
        return None, None

# Function to determine the win method
def determine_win_method(winner_data, loser_data):
    ko_diff = winner_data['KoTkoPercentage'] - loser_data['KoTkoPercentage']
    sub_diff = winner_data['SubmissionPercentage'] - loser_data['SubmissionPercentage']
    dec_diff = winner_data['DecisionPercentage'] - loser_data['DecisionPercentage']

    if ko_diff > sub_diff and ko_diff > dec_diff:
        return 'knockout'
    elif sub_diff > ko_diff and sub_diff > dec_diff:
        return 'submission'
    else:
        return 'decision'

# Create UI elements
weight_classes = dataset['weightClass'].unique()
fighters_by_weight_class = {wc: original_dataset[original_dataset['weightClass'] == wc] for wc in weight_classes}

weight_class_dropdown = widgets.Dropdown(
    options=weight_classes,
    description='Weight Class:'
)

fighter1_dropdown = widgets.Dropdown(
    options=[],
    description='Fighter 1:'
)

fighter2_dropdown = widgets.Dropdown(
    options=[],
    description='Fighter 2:'
)

# Ensure that fighter dropdowns are populated with the default weight class fighters when the code is first executed
def initialize_fighter_dropdowns():
    selected_weight_class = weight_class_dropdown.value  # Get the default value of the weight class dropdown
    fighters = fighters_by_weight_class[selected_weight_class]['name'].values
    fighter1_dropdown.options = fighters
    fighter2_dropdown.options = fighters

# Call the function to initialize the fighter dropdowns

initialize_fighter_dropdowns()

def update_fighter_dropdowns(change):
    selected_weight_class = change['new']
    fighters = fighters_by_weight_class[selected_weight_class]['name'].values
    fighter1_dropdown.options = fighters
    fighter2_dropdown.options = fighters

weight_class_dropdown.observe(update_fighter_dropdowns, names='value')

predict_button = widgets.Button(description='Predict')
output = widgets.Output()

def is_valid_url(url):
    try:
        result = requests.get(url)
        return result.status_code == 200
    except:
        return False

def display_prediction_result(fighter1_data, fighter2_data, prediction_result, win_method):
    with output:
        clear_output()

        fighter1_img = widgets.Image(value=requests.get(fighter1_data['image']).content) if is_valid_url(fighter1_data['image']) else widgets.Label("No Image")
        fighter2_img = widgets.Image(value=requests.get(fighter2_data['image']).content) if is_valid_url(fighter2_data['image']) else widgets.Label("No Image")

        fighter1_info = widgets.HTML(f"""
            <h3>{fighter1_data['name']}</h3>
            <p>Record: {fighter1_data['wins']} wins, {fighter1_data['losses']} losses, {fighter1_data['draws']} draws</p>
            <p>Height: {fighter1_data['height']} inches</p>
            <p>Weight: {fighter1_data['weight']} lbs</p>
            <p>Reach: {fighter1_data['reach']} inches</p>
            <p>KO/TKO Percentage: {fighter1_data['KoTkoPercentage']*100}%</p>
            <p>Submission Percentage: {fighter1_data['SubmissionPercentage']*100}%</p>
            <p>Decision Percentage: {fighter1_data['DecisionPercentage']*100}%</p>
        """)

        fighter2_info = widgets.HTML(f"""
            <h3>{fighter2_data['name']}</h3>
            <p>Record: {fighter2_data['wins']} wins, {fighter2_data['losses']} losses, {fighter2_data['draws']} draws</p>
            <p>Height: {fighter2_data['height']} inches</p>
            <p>Weight: {fighter2_data['weight']} lbs</p>
            <p>Reach: {fighter2_data['reach']} inches</p>
            <p>KO/TKO Percentage: {fighter2_data['KoTkoPercentage']*100}%</p>
            <p>Submission Percentage: {fighter2_data['SubmissionPercentage']*100}%</p>
            <p>Decision Percentage: {fighter2_data['DecisionPercentage']*100}%</p>
        """)

        display(widgets.HBox([widgets.VBox([fighter1_img, fighter1_info]), widgets.VBox([fighter2_img, fighter2_info])]))

        if prediction_result:
            winner_data = fighters_by_weight_class[weight_class_dropdown.value][fighters_by_weight_class[weight_class_dropdown.value]['name'] == prediction_result].iloc[0]
            winner_img = widgets.Image(value=requests.get(winner_data['image']).content) if is_valid_url(winner_data['image']) else widgets.Label("No Image")
            winner_info = widgets.HTML(f"""
                <h3>Predicted Winner: {prediction_result}</h3>
                <p>{prediction_result} wins by {win_method}</p>
            """)
            display(widgets.HBox([winner_img, winner_info]))

def on_predict_button_clicked(b):
    fighter1_name = fighter1_dropdown.value
    fighter2_name = fighter2_dropdown.value

    with output:
        clear_output()

        # Check if the same fighter is selected in both dropdowns
        if fighter1_name == fighter2_name:
            display(widgets.HTML(f"<h3 style='color: red;'>A fighter cannot fight himself in the ring, but only in mind!</h3>"))
        else:
            # Proceed with prediction if different fighters are selected
            fighter1_data, _ = get_fighter_stats(fighter1_name)
            fighter2_data, _ = get_fighter_stats(fighter2_name)

            prediction_result, win_method = predict_fight_outcome(fighter1_name, fighter2_name)

            display_prediction_result(fighter1_data, fighter2_data, prediction_result, win_method)

predict_button.on_click(on_predict_button_clicked)

display(weight_class_dropdown, fighter1_dropdown, fighter2_dropdown, predict_button, output)

Dropdown(description='Weight Class:', options=('Heavyweight Division', 'Welterweight Division', 'Middleweight …

Dropdown(description='Fighter 1:', options=('Shamil Abdurakhimov', 'Scott Adams', 'Juan Adams', 'Mostapha Al T…

Dropdown(description='Fighter 2:', options=('Shamil Abdurakhimov', 'Scott Adams', 'Juan Adams', 'Mostapha Al T…

Button(description='Predict', style=ButtonStyle())

Output()